# A Gesture-Based Tool For Sterile Browsing Of Radiology Images

In this project we have used Convolutional Neural Network to first train the model on the images of different hand gestures, like showing numbers with fingers as 0,1,2,3,4,5. Then we made a web portal using Flask where user can input any image on which he wants to perform the operations. After uploading the image, our portal uses the integrated webcam to capture the video frame using OpenCV. The gesture captured in the video frame is compared with the Pre-trained model and the gesture is identified. If the prediction is 0 - then images is converted into rectangle, 1 - image is Resized into (200,200), 2 - image is rotated by -45॰, 3 - image is blurred , 4 - image is Resized into (400,400) , 5 - image is converted into grayscale.

# Model Building

## Importing libraries

In [3]:
# This library helps add support for large, multi-dimensional arrays and matrices
import numpy as np
#open source used for both ML and DL for computation
import tensorflow as tf
#it is a plain stack of layers
from tensorflow.keras.models import Sequential 
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten, Dropout
#Faltten-used fot flattening the input or change the dimension, MaxPooling2D-for downsampling the image for Convolutional layer
from tensorflow.keras.layers import Convolution2D,MaxPooling2D 
#Its used for different augmentation of the image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Augmenting the data

In [4]:
#setting parameter for Image Data agumentation to the traing data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

## Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'C:\Users\chandu\Desktop\IBM_Project\Dataset\train',
                                            target_size=(64, 64),
                                            batch_size=3,
                                            color_mode='grayscale',
                                            class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'C:\Users\chandu\Desktop\IBM_Project\Dataset\test',
                                          target_size=(64, 64),
                                          batch_size=3,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


## Model Creation

In [7]:
# Initializing the CNN
model = Sequential()

In [8]:
# First convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [9]:
# Second convolution layer and pooling
model.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
# Flattening the layers i.e. input layer
model.add(Flatten())

In [11]:
# Adding a fully connected layer, i.e. Hidden Layer
model.add(Dense(units=512 , activation='relu'))

In [12]:
# softmax for categorical analysis, Output Layer
model.add(Dense(units=6, activation='softmax')) 

In [13]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

## Model Compilation

In [14]:
# Compiling the CNN
# categorical_crossentropy for more than 2
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Model fitting

In [15]:
# It will generate packets of train and test data for training
model.fit_generator(x_train,
                    steps_per_epoch = 594/3 , 
                    epochs = 25, 
                    validation_data = x_test,
                    validation_steps = 30/3 )

C:\Users\chandu\AppData\Local\Temp/ipykernel_16664/804983804.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


Epoch 1/25
198/198 [==============================] - 15s 64ms/step - loss: 1.3510 - accuracy: 0.4663 - val_loss: 0.6526 - val_accuracy: 0.7333
Epoch 2/25
198/198 [==============================] - 6s 32ms/step - loss: 0.5420 - accuracy: 0.7862 - val_loss: 0.5013 - val_accuracy: 0.8333
Epoch 3/25
198/198 [==============================] - 6s 32ms/step - loss: 0.4282 - accuracy: 0.8350 - val_loss: 0.3105 - val_accuracy: 0.8667
Epoch 4/25
198/198 [==============================] - 7s 34ms/step - loss: 0.3142 - accuracy: 0.8788 - val_loss: 0.3412 - val_accuracy: 0.9333
Epoch 5/25
198/198 [==============================] - 7s 33ms/step - loss: 0.1804 - accuracy: 0.9411 - val_loss: 0.1762 - val_accuracy: 0.9667
Epoch 6/25
198/198 [==============================] - 7s 33ms/step - loss: 0.1319 - accuracy: 0.9512 - val_loss: 0.2698 - val_accuracy: 0.9000
Epoch 7/25
198/198 [==============================] - 7s 33ms/step - loss: 0.1519 - accuracy: 0.9411 - val_loss: 0.3620 - val_accuracy: 0.933

## Saving model

In [16]:
# Save the model
model.save('gesture.h5')

In [17]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)